In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import holoviews as hv
import panel as pn
import datetime as dt
from get_data import get_urls_mod, get_urls_obs, get_model_data, get_obs_data

In [ ]:
hv.extension("bokeh")
pn.extension()

In [ ]:
def calc_windspeed(u_a,v_a):
    #only for 10-m
    u_a.columns = ['comp']
    v_a.columns = ['comp']
    windspeed_calc = lambda u,v: (np.sqrt(u*u+v*v))
    windspeed = u_a.combine(v_a,windspeed_calc)
    return windspeed

def calc_ustar(tau_u,tau_v):
    #only for 10-m
    tau_u.columns = ['comp']
    tau_v.columns = ['comp']
    u_star_calc = lambda u,v: (np.sqrt(np.sqrt(u*u + v*v)))
    ustar = tau_u.combine(tau_v,u_star_calc)
    
    return ustar

def calc_scaledHeatFlux(hf,wspeed):
    hf.columns = ['comp']
    wspeed.columns = ['comp']
    scaledHF_calc = lambda u,v: (u/v)
    scaledHF = hf.combine(wspeed, scaledHF_calc)
    return scaledHF

def calc_gradient(ref, s):
    ref.columns = ['comp']
    s.columns = ['comp']
    calc_grad = lambda u,v: (u-v)
    grad = ref.combine(s,calc_grad)
    return grad

In [ ]:
#selection of site
site_name = pn.widgets.Select(
    name="Site", options=['sodankyla','tiksi'], margin=(0, 20, 0, 0)
)

#selection of model
model_name = pn.widgets.Select(
    name="model", options=['AROME-Arctic','ifs-ecmwf'], margin=(0, 20, 0, 0)
)
#selection of case to plot
case = pn.widgets.Select(
    name="Case", options={"Ustar vs. Windspeed":"case1","Scaled heat flux vs. temperature gradient":"case2"}, margin=(0, 20, 0, 0)
)

#user selection start and end date only sop 1 as of right now
dates = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2018, 2, 1), end=dt.datetime(2018, 3, 31),
    value=(dt.datetime(2018, 2, 1), dt.datetime(2018, 2, 4))
)

#user selection start time for model 
start_time = pn.widgets.Select(
    name="Start time", options=['00','12'], margin=(0, 20, 0, 0)
)

#user selection of which day should be fetched from the model files
#for concat
concat_day = pn.widgets.Select(
    name="Model day selected", options={1:0,2:1,3:2}, margin=(0,20,0,0)
)

#TO DO: add dependecy so that 1-1 toggles line reg off and vice versa
line_reg = pn.widgets.Checkbox(name='Linear regression')
one_one = pn.widgets.Checkbox(name='1 to 1 line')

In [ ]:
def scatter2(x,y):
    plot =  hv.Points((x['comp'],y['comp']))
    return plot
    

In [ ]:
@pn.depends(site_name, model_name,case,dates, concat_day,start_time)
def scatter_2(site_name,model_name,case, dates, concat_day,start_time):

    if case == "case1":
        #ustar vs. windspeed
        variables = ['tauu','tauv','uas','vas']
        #=====================
        #get data
        urls_mod = get_urls_mod(dates[0],dates[1],start_time,model_name,site_name,variables,concat_day)
        data_mod = get_model_data("concatenated",urls_mod,concat_day)
        #====================
        #calculate relevant things
        wspeed = calc_windspeed(data_mod['uas'].to_dataframe(),data_mod['vas'].to_dataframe())
        ustar = calc_ustar(data_mod['tauu'].to_dataframe(),data_mod['tauv'].to_dataframe())
        #====================
        #plot
        plot = scatter2(wspeed,ustar)
        
        plot.opts(title="Scatter 2: "+"Ustar vs. windspeed"+'\n'+' '+model_name+' '+site_name,
        ylabel="Ustar",
        xlabel="Windspeed",
        width=400,
        height=400
        ) 
    elif case == "case2":
        #scaled heat flux and temperature gradient
        variables = ['hfss','uas','vas','ts','tas']
        #====================
        #get data
        urls_mod = get_urls_mod(dates[0],dates[1],start_time,model_name,site_name,variables,concat_day)
        data_mod = get_model_data("concatenated",urls_mod,concat_day)
        #====================
        #calculate relevant things
        wspeed = calc_windspeed(data_mod['uas'].to_dataframe(),data_mod['vas'].to_dataframe())
        scHF = calc_scaledHeatFlux(data_mod['hfss'].to_dataframe(),wspeed)
        t_grad = calc_gradient(data_mod['tas'].to_dataframe(),data_mod['ts'].to_dataframe()) 
        #====================
        #plot
        plot = scatter2(t_grad,scHF)
        plot.opts(title="Scatter 2: "+"Scaled heat flux vs."+'\n'+" temperature gradient"+'\n'+' '+model_name+' '+site_name,
        ylabel="Scaled heat flux",
        xlabel="Temperature gradient",
        width=400,
        height=400
        )
        
    elif case == "case3":
        #scaled water vapor flux and humidity gradient
        #hfls scaled with windspeed in same way
        variables = ['huss','hus','hfls','uas','vas']
        #lowest model level for hus minus 2m (huss)
        #get data
        urls = get_urls(dates[0],dates[1],start_time,model_name,site_name,variables,concat_day)
        data_mod = get_model_data("concatenated",urls[0],concat_day)
        #===========================
        wspeed = calc_windspeed(data_mod['uas'],data_mod['vas'])
        scHF = calc_scaledHeatFlux(data_mod['hfls'],wspeed)
        lowest_level_h = data_mod['hus'] #take lowest model level
        h_grad = calc_gradient(lowest_level_h,data_mod['huss'])
        plot = scatter2(h_grad,scHF) 
        
    
    return plot

In [ ]:
pn.Column("scatter 2 sop 1", scatter_2,pn.Row(pn.Column(case,site_name,model_name),pn.Column(dates, concat_day,start_time)),width_policy="max").servable()
